The purpose of GST's fiducials is to generate an informationally-complete set of states and measurements. However, it's possible to _reduce_ the number of fiducials necessary to do so.

In this tutorial, we use the `pygsti.alg.find_sufficient_fiducial_pairs` function to reduce the number of fiducials necessary for GST.

In [ ]:
from __future__ import print_function

In [ ]:
import pygsti
import json

In [ ]:
# Follow Algorithm tutorial to generate LGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of LGST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))

In [ ]:
#Get sufficient set of fiducial pairs, meaning that with these fidicual pairs and the given set of germs,
# the number of gateset parameters which are amplified when all pairs are used are also amplified when using
# the returned subset.
fidPairs = pygsti.alg.find_sufficient_fiducial_pairs(gs_target, fiducialList, fiducialList, germList, verbosity=1)
print(fidPairs)

In [ ]:
#Test a specific set of fiducial pairs: see how many gateset parameter are amplified
#FPR.find_sufficient_fiducial_pairs(gs_target, fiducialList, germList, testPairList=[(0,0),(0,1),(1,0)], verbosity=4)

In [ ]:
lsgstListOfLists = pygsti.construction.make_lsgst_lists(gs_target.gates.keys(), fiducialList, fiducialList,
                                                        germList, maxLengthList, fidPairs)

gs_lsgst_list = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )

In [ ]:
# Compute a few additional quantities needed to generate the report
Ls = maxLengthList
gateStrDict = { (L,germ):pygsti.construction.repeat_with_max_length(germ,L,False) for L in Ls for germ in germList }

#remove duplicates by replacing duplicate strings with None
runningList = []
for L in Ls:
    for germ in germList:
        if gateStrDict[(L,germ)] in runningList:
            gateStrDict[(L,germ)] = None
        else: runningList.append( gateStrDict[(L,germ)] )

In [ ]:
#optimize each gateset to the target
gs_lsgst_list = [ pygsti.optimize_gauge(gs, "target", targetGateset=gs_target) for gs in gs_lsgst_list ]

In [ ]:
res = pygsti.report.Results()
res.init_Ls_and_germs("chi2", gs_target, ds, gs_clgst, maxLengthList, germList,
                    gs_lsgst_list, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False, fidPairs)

res.create_full_report_pdf(filename="tutorial_files/Example_report_FR.pdf", verbosity=2)